# HW1
Juan Vila

In [274]:
#Importing Packeges
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
from sodapy import Socrata
from shapely.geometry import shape
from shapely.geometry import Point
import censusgeocode as cg

#fix parameters
#keys
census_key = "59bc19643ed2a3eaabf9848b92e912c3c65b0ab5"
chicago_api_key = "jx6Btc30ora3Vx9oia6r2H3HG"
user_chicago_api = "jiv@uchicago.edu"
pass_chicago_api = "Jiv.12345678"
#data
crime_key = "6zsd-86xi"
community_code = "igwz-8jzy"
block_code = "bt9m-d2mf"
census_data  = "https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E,B17020_002E,B19051_001E&for=tract:*&in=state:17%20county:031"
time_to_work_data  = "https://api.census.gov/data/2017/acs/acs5?get=NAME,B08303_001E,B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E&for=tract:*&in=state:17%20county:031"




## Question 1

### Question 1 a

In [164]:
#Get Data:
#Functions
def get_crime_data(data_key, crime=True):
    '''
    Description: This function its commnicate to the City Chicago Open Data servers and extract information
    Input:
        Data_key: code that identify the dataset in the chicago open data dataset.
        crime:(True) it's for struact specificallly the crime data.
    Output:
    '''
    client = Socrata("data.cityofchicago.org", chicago_api_key, username=user_chicago_api,
                     password=pass_chicago_api, timeout=1000)
    if crime:
        results = client.get(data_key, limit = 7000000, where='year>2016 AND year<2019')
    else:
        results = client.get(data_key, limit = 1000)
    rv = pd.DataFrame.from_records(results)
    return rv

def get_json_data(key, limit, geo_pandas=True):
    '''
    Description: This function its commnicate to the City Chicago Open Data servers and extract information and
                convert to a pandas frame or geopandas frme.
    Input: 
        key(str): code that identify the dataset in the chicago open data dataset.
        limit(int): limit of observations that the we can download from the original json file.
        geo_pandas(boolean): if its true use geo pandas instead of pandas
    Output:
        Dataframe.
    '''
    http = "https://data.cityofchicago.org/resource/"
    lim = "?$limit=" + str(limit)
    json_str = ".json"
    request_str = http + key + json_str + lim
    r = requests.get(request_str)
    r.raise_for_status()
    data = r.json()

    if geo_pandas:
        for d in data:
            d['the_geom'] = shape(d['the_geom'])
        rv = gpd.DataFrame(data).set_geometry('the_geom')
    else:
        rv = pd.DataFrame(data)
    return rv

#Getting data   
gdf_blocks_chicago = get_geojson_data(block_code, 50000)
df_crime = get_crime_data(crime_key)
df_comm_chicago = get_crime_data(community_code, False)
df_crime = df_crime.dropna(subset=['latitude', 'longitude'])
df_crime.latitude = df_crime.latitude.astype(dtype=np.float64)
df_crime.longitude = df_crime.longitude.astype(dtype=np.float64)
df_crime = df_crime.merge(df_comm_chicago, left_on='community_area', right_on='area_num_1')

46357

### Question 1 b

In [222]:
#Number of arrest by year
tab1=pd.crosstab(index=df_crime["arrest"],columns=df_crime["year"], margins=True)
print(tab1)

year      2017    2018     All
arrest                        
False   212919  211160  424079
True     52063   52380  104443
All     264982  263540  528522


In [223]:
#% of arrest by year
tab1/tab1.loc["All"]

year,2017,2018,All
arrest,,,
False,0.803523,0.801245,0.802387
True,0.196477,0.198755,0.197613
All,1.000000,1.000000,1.000000


In [224]:
#total of incidnets and arrests by year and community
tab5 = pd.crosstab(index=df_crime["community"], columns=[df_crime["year"],df_crime["arrest"]],margins=True) 
tab5

year                      2017           2018            All
arrest                   False   True   False   True        
community                                                   
ALBANY PARK               2208    242    2120    272    4842
ARCHER HEIGHTS             722    114     706    105    1647
ARMOUR SQUARE              851    159     860    136    2006
ASHBURN                   2178    288    2137    246    4849
AUBURN GRESHAM            5711   1751    5528   1785   14775
AUSTIN                   11329   3945   11127   3819   30220
AVALON PARK               1067    204     948    203    2422
AVONDALE                  2015    301    2049    250    4615
BELMONT CRAGIN            3777    767    3625    703    8872
BEVERLY                    775    102     810     88    1775
BRIDGEPORT                1306    178    1280    211    2975
BRIGHTON PARK             1767    426    1782    382    4357
BURNSIDE                   292     73     285     96     746
CALUMET HEIGHTS           1159    188    1136    239    2722
CHATHAM                   4937   1410    4940   1523   12810
CHICAGO LAWN              4643   1053    4649    987   11332
CLEARING                   775    106     849     95    1825
DOUGLAS                   2025    553    2194    478    5250
DUNNING                   1273    226    1309    181    2989
EAST GARFIELD PARK        3304   1408    3582   1298    9592
EAST SIDE                  963    241     854    237    2295
EDGEWATER                 2189    399    2344    377    5309
EDISON PARK                231     37     229     21     518
ENGLEWOOD                 4408   1487    4637   1538   12070
FOREST GLEN                499     40     450     36    1025
FULLER PARK                700    174     634    197    1705
GAGE PARK                 1850    430    1668    385    4333
GARFIELD RIDGE            1587    312    1673    391    3963
GRAND BOULEVARD           2814    582    2670    535    6601
GREATER GRAND CROSSING    5103   1315    4857   1440   12715
...                        ...    ...     ...    ...     ...
NEAR SOUTH SIDE           1597    297    1543    265    3702
NEAR WEST SIDE            7673   1267    7910   1353   18203
NEW CITY                  3485   1121    3228    959    8793
NORTH CENTER              1251    143    1178    120    2692
NORTH LAWNDALE            5721   3253    5647   3589   18210
NORTH PARK                 818    135     885    106    1944
NORWOOD PARK               952    129    1048    104    2233
OAKLAND                    631     39     599     39    1308
OHARE                     1165    471    1230    344    3210
PORTAGE PARK              2835    493    2754    409    6491
PULLMAN                    731    356     827    369    2283
RIVERDALE                 1038    301    1042    302    2683
ROGERS PARK               3444    641    3118    635    7838
ROSELAND                  5206   1682    5133   1715   13736
SOUTH CHICAGO             3499   1034    3284   1007    8824
SOUTH DEERING             1338    405    1314    421    3478
SOUTH LAWNDALE            3563    926    3458   1049    8996
SOUTH SHORE               6915   1673    6843   1745   17176
UPTOWN                    2896    617    2978    599    7090
WASHINGTON HEIGHTS        2540    514    2563    590    6207
WASHINGTON PARK           2269    432    2318    465    5484
WEST ELSDON                881    128     866    142    2017
WEST ENGLEWOOD            5064   1838    5256   1775   13933
WEST GARFIELD PARK        3310   2029    3528   2099   10966
WEST LAWN                 1487    370    1483    285    3625
WEST PULLMAN              3085    874    3171    845    7975
WEST RIDGE                3105    443    3060    426    7034
WEST TOWN                 7390    829    6408    762   15389
WOODLAWN                  3062    739    2696    746    7243
All                     212919  52063  211160  52380  528522

[78 rows x 5 columns]

In [225]:
#top 10 communities by number of incidents in 2018
tab2_2018 = df_crime[(df_crime['year']  == '2018')].groupby(['community'])['arrest'].count().nlargest(10)
tab2_2018

community
AUSTIN             14946
NEAR NORTH SIDE    12845
LOOP               10666
NEAR WEST SIDE      9263
NORTH LAWNDALE      9236
SOUTH SHORE         8588
HUMBOLDT PARK       7983
AUBURN GRESHAM      7313
WEST TOWN           7170
WEST ENGLEWOOD      7031
Name: arrest, dtype: int64

In [160]:
#top 10 communities by number of incidents in 2017
tab2_2017 = df_crime[(df_crime['year']  == '2017')].groupby(['community'])['arrest'].count().nlargest(10)
tab2_2017

community
AUSTIN             15274
NEAR NORTH SIDE    12115
LOOP               10474
NORTH LAWNDALE      8973
NEAR WEST SIDE      8940
SOUTH SHORE         8588
WEST TOWN           8219
HUMBOLDT PARK       8006
AUBURN GRESHAM      7462
WEST ENGLEWOOD      6902
Name: arrest, dtype: int64

In [227]:
#top 10 communities by number of arrests in 2018
tab3_2018 = df_crime[(df_crime['year']  == '2018')].groupby(['community'])['arrest'].sum().nlargest(10)
tab3_2018

community
AUSTIN                3819.0
NORTH LAWNDALE        3589.0
HUMBOLDT PARK         2846.0
WEST GARFIELD PARK    2099.0
NEAR NORTH SIDE       2098.0
AUBURN GRESHAM        1785.0
WEST ENGLEWOOD        1775.0
SOUTH SHORE           1745.0
ROSELAND              1715.0
LOOP                  1612.0
Name: arrest, dtype: float64

In [228]:
#top 10 communities by number of arrests in 2017
tab3_2017 = df_crime[(df_crime['year']  == '2017')].groupby(['community'])['arrest'].sum().nlargest(10)
tab3_2017

community
AUSTIN                3945.0
NORTH LAWNDALE        3253.0
HUMBOLDT PARK         2313.0
NEAR NORTH SIDE       2134.0
WEST GARFIELD PARK    2029.0
WEST ENGLEWOOD        1838.0
AUBURN GRESHAM        1751.0
ROSELAND              1682.0
SOUTH SHORE           1673.0
LOOP                  1622.0
Name: arrest, dtype: float64

In [229]:
#Top 10 communities when a call generate an arrest as porcentange of the total of incidents in 2018
crime_2018 = df_crime[(df_crime['year']  == '2018')]
tab4_2018=pd.crosstab(index=crime_2018["community"], columns = crime_2018['arrest'], margins=True)
tab4_2018.columns = ["Non-Arrested","Arrested", "Total"]
tab4_2018 = tab4_2018/tab4_2018.loc["All","Total"]
tab4_2018 = tab4_2018.div(tab4_2018["Total"], axis=0)
tab4_2018.sort_values("Arrested",ascending=False).head(10)

,Non-Arrested,Arrested,Total
community,,,
NORTH LAWNDALE,0.611412,0.388588,1.0
WEST GARFIELD PARK,0.626977,0.373023,1.0
HUMBOLDT PARK,0.643492,0.356508,1.0
PULLMAN,0.691472,0.308528,1.0
EAST GARFIELD PARK,0.734016,0.265984,1.0
AUSTIN,0.744480,0.255520,1.0
WEST ENGLEWOOD,0.747547,0.252453,1.0
BURNSIDE,0.748031,0.251969,1.0
ROSELAND,0.749562,0.250438,1.0


In [271]:
#Top 10 communities when a call generate an arrest as porcentange of the total of incidents in 2017
crime_2017 = df_crime[(df_crime['year']  == '2017')]
tab4_2017 = pd.crosstab(index=crime_2017["community"], columns = crime_2017['arrest'], margins=True)
tab4_2017.columns = ["Non-Arrested","Arrested", "Total"]
tab4_2017 = tab4_2017/tab4_2017.loc["All","Total"]
tab4_2017 = tab4_2017.div(tab4_2017["Total"], axis=0)
tab4_2017.sort_values("Arrested",ascending=False).head(10)

,Non-Arrested,Arrested,Total
community,,,
WEST GARFIELD PARK,0.619966,0.380034,1.0
NORTH LAWNDALE,0.637468,0.362532,1.0
PULLMAN,0.672493,0.327507,1.0
EAST GARFIELD PARK,0.701188,0.298812,1.0
HUMBOLDT PARK,0.711092,0.288908,1.0
OHARE,0.712103,0.287897,1.0
WEST ENGLEWOOD,0.733700,0.266300,1.0
AUSTIN,0.741718,0.258282,1.0
ENGLEWOOD,0.747752,0.252248,1.0


In [230]:
tab6_2018 = df_crime[(df_crime['year']=='2018')].groupby(['primary_type'])['arrest'].count().nlargest(32)
tab6_2018

primary_type
THEFT                                64025
BATTERY                              49719
CRIMINAL DAMAGE                      27700
ASSAULT                              20341
DECEPTIVE PRACTICE                   17862
OTHER OFFENSE                        16970
NARCOTICS                            12797
BURGLARY                             11687
MOTOR VEHICLE THEFT                   9964
ROBBERY                               9680
CRIMINAL TRESPASS                     6881
WEAPONS VIOLATION                     5444
OFFENSE INVOLVING CHILDREN            2174
CRIM SEXUAL ASSAULT                   1588
PUBLIC PEACE VIOLATION                1362
INTERFERENCE WITH PUBLIC OFFICER      1305
SEX OFFENSE                           1061
PROSTITUTION                           717
HOMICIDE                               586
ARSON                                  372
LIQUOR LAW VIOLATION                   265
GAMBLING                               201
STALKING                               20

In [231]:
tab6_2017 = df_crime[(df_crime['year']=='2017')].groupby(['primary_type'])['arrest'].count().nlargest(32)
tab6_2017

primary_type
THEFT                                63586
BATTERY                              49137
CRIMINAL DAMAGE                      28955
ASSAULT                              19251
DECEPTIVE PRACTICE                   17768
OTHER OFFENSE                        16965
BURGLARY                             12947
ROBBERY                              11869
NARCOTICS                            11475
MOTOR VEHICLE THEFT                  11367
CRIMINAL TRESPASS                     6799
WEAPONS VIOLATION                     4682
OFFENSE INVOLVING CHILDREN            2145
CRIM SEXUAL ASSAULT                   1541
PUBLIC PEACE VIOLATION                1496
INTERFERENCE WITH PUBLIC OFFICER      1083
SEX OFFENSE                            939
PROSTITUTION                           734
HOMICIDE                               676
ARSON                                  444
GAMBLING                               191
LIQUOR LAW VIOLATION                   191
KIDNAPPING                             19

In [273]:
#Top 10 types of crime when a call generate an arrest as porcentange of the total of incidents in 2018
crime_2018 = df_crime[(df_crime['year']  == '2018')]
tab7_2018=pd.crosstab(index=crime_2018['primary_type'], columns = crime_2018['arrest'], margins=True)
tab7_2018.columns = ["Non-Arrested","Arrested", "Total"]
tab7_2018 = tab7_2018/tab7_2018.loc["All","Total"]
tab7_2018 = tab7_2018.div(tab7_2018["Total"], axis=0)
tab7_2018.sort_values("Arrested",ascending=False).head(10)

,Non-Arrested,Arrested,Total
primary_type,,,
LIQUOR LAW VIOLATION,0.000000,1.000000,1.0
PROSTITUTION,0.000000,1.000000,1.0
GAMBLING,0.000000,1.000000,1.0
PUBLIC INDECENCY,0.000000,1.000000,1.0
NARCOTICS,0.000469,0.999531,1.0
CONCEALED CARRY LICENSE VIOLATION,0.027027,0.972973,1.0
INTERFERENCE WITH PUBLIC OFFICER,0.045977,0.954023,1.0
OBSCENITY,0.186047,0.813953,1.0
WEAPONS VIOLATION,0.302351,0.697649,1.0


In [292]:
#Top 10 types of crime when a call generate an arrest as porcentange of the total of incidents in 2017
crime_2017 = df_crime[(df_crime['year']  == '2017')]
tab7_2017=pd.crosstab(index=crime_2017['primary_type'], columns = crime_2017['arrest'], margins=True)
tab7_2017.columns = ["Non-Arrested","Arrested", "Total"]
tab7_2017 = tab7_2017/tab7_2017.loc["All","Total"]
tab7_2017 = tab7_2017.div(tab7_2017["Total"], axis=0)
tab7_2017.sort_values("Arrested",ascending=False).head(10)

,Non-Arrested,Arrested,Total
primary_type,,,
PROSTITUTION,0.000000,1.000000,1.0
GAMBLING,0.000000,1.000000,1.0
LIQUOR LAW VIOLATION,0.000000,1.000000,1.0
PUBLIC INDECENCY,0.000000,1.000000,1.0
NARCOTICS,0.000436,0.999564,1.0
CONCEALED CARRY LICENSE VIOLATION,0.043478,0.956522,1.0
INTERFERENCE WITH PUBLIC OFFICER,0.048938,0.951062,1.0
OBSCENITY,0.212500,0.787500,1.0
WEAPONS VIOLATION,0.214865,0.785135,1.0


## Question 2

In [52]:
#Spatial merging.
geometry = [Point(xy) for xy in zip(df_crime.longitude, df_crime.latitude)]
df_crime = df_crime.drop(['longitude', 'latitude'], axis=1)
crs = {'init': 'epsg:4326'}
gdf_crime = gpd.GeoDataFrame(df_crime, crs=crs, geometry=geometry)
crime_with_blocks = gpd.sjoin(gdf_blocks_chicago, gdf_crime, how="inner", op='intersects')

In [130]:
#Importing census data
#income and poverty
df_census = pd.read_json(census_data)
df_census.columns = df_census.iloc[0]
df_census = df_census.reindex(df_census.index.drop(0))
df_census.B17020_002E = df_census.B17020_002E.astype(dtype=np.float64)#number of poor people
df_census.B01001_001E = df_census.B01001_001E.astype(dtype=np.float64)#population
df_census.B19051_001E = df_census.B19051_001E.astype(dtype=np.float64)#average earnings
df_census['pseudo_pov_rate'] = df_census['B17020_002E']/df_census['B01001_001E']

#time to work
df_ttw = pd.read_json(time_to_work_data)
df_ttw.columns = df_ttw.iloc[0]
df_ttw = df_ttw.reindex(df_ttw.index.drop(0))
df_ttw.B08303_001E = df_ttw.B08303_001E.astype(dtype=np.float64)#total
df_ttw.B08303_002E = df_ttw.B08303_002E.astype(dtype=np.float64) * 2.5
df_ttw.B08303_003E = df_ttw.B08303_003E.astype(dtype=np.float64) * 7.5
df_ttw.B08303_004E = df_ttw.B08303_004E.astype(dtype=np.float64) * 12.5
df_ttw.B08303_005E = df_ttw.B08303_005E.astype(dtype=np.float64) * 17.5
df_ttw.B08303_006E = df_ttw.B08303_006E.astype(dtype=np.float64) * 22.5
df_ttw.B08303_007E = df_ttw.B08303_007E.astype(dtype=np.float64) * 27.5
df_ttw.B08303_008E = df_ttw.B08303_008E.astype(dtype=np.float64) * 32.5
df_ttw.B08303_009E = df_ttw.B08303_009E.astype(dtype=np.float64) * 37.5
df_ttw.B08303_010E = df_ttw.B08303_010E.astype(dtype=np.float64) * 42.5
df_ttw.B08303_011E = df_ttw.B08303_011E.astype(dtype=np.float64) * 52.5
df_ttw.B08303_012E = df_ttw.B08303_012E.astype(dtype=np.float64) * 75
df_ttw.B08303_013E = df_ttw.B08303_013E.astype(dtype=np.float64) * 90

df_ttw['average_time'] = df_ttw.B08303_002E + df_ttw.B08303_003E + df_ttw.B08303_004E + df_ttw.B08303_005E + df_ttw.B08303_006E + df_ttw.B08303_007E + df_ttw.B08303_008E + df_ttw.B08303_009E + df_ttw.B08303_010E + df_ttw.B08303_011E + df_ttw.B08303_012E + df_ttw.B08303_013E
df_ttw['average_time'] = df_ttw['average_time']/df_ttw['B08303_001E']

#merging with crime data
crime_with_blocks = crime_with_blocks.merge(df_ttw, left_on='tractce10', right_on='tract')
crime_with_blocks = crime_with_blocks.merge(df_census, left_on='tractce10', right_on='tract')




### Q2-p1

In [139]:
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY')].pseudo_pov_rate.describe()

count    98285.000000
mean         0.282587
std          0.143196
min          0.004437
25%          0.166633
50%          0.279943
75%          0.384096
max          0.736909
Name: pseudo_pov_rate, dtype: float64

In [148]:
crime_with_blocks[(crime_with_blocks['primary_type'] != 'BATTERY')].pseudo_pov_rate.describe()

count    425786.000000
mean          0.244890
std           0.143551
min           0.004437
25%           0.125938
50%           0.229087
75%           0.342600
max           0.736909
Name: pseudo_pov_rate, dtype: float64

In [141]:
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY')].average_time.describe()

count    98285.000000
mean        40.434745
std          5.899585
min         20.351382
25%         36.986375
50%         40.958333
75%         43.986920
max         60.885762
Name: average_time, dtype: float64

In [149]:
crime_with_blocks[(crime_with_blocks['primary_type'] != 'BATTERY')].average_time.describe() 

count    425786.000000
mean         38.954815
std           6.399348
min          20.351382
25%          35.227273
50%          39.656411
75%          43.424339
max          60.885762
Name: average_time, dtype: float64

In [144]:
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY')].B19051_001E.describe()

count    98617.000000
mean      1451.545545
std       1081.787312
min          0.000000
25%        816.000000
50%       1275.000000
75%       1790.000000
max      11012.000000
Name: B19051_001E, dtype: float64

In [147]:
crime_with_blocks.B19051_001E[(crime_with_blocks['primary_type'] != 'BATTERY')].describe()

count    428595.000000
mean       1633.186626
std        1328.136587
min           0.000000
25%         883.000000
50%        1370.000000
75%        1902.000000
max       11012.000000
Name: B19051_001E, dtype: float64

Answer 1 part 2: we can see that for the primary type battery for both years, that in general it's occurs in more poorer blocks, with less income and more time in traveling to the different workplace where people works. This means that in general battery occurs in lower income neigborhoods. 

### Q2-p2

In [151]:
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE')].pseudo_pov_rate.describe()

count    1262.000000
mean        0.324374
std         0.131785
min         0.026226
25%         0.229087
50%         0.332072
75%         0.405901
max         0.736909
Name: pseudo_pov_rate, dtype: float64

In [152]:
crime_with_blocks[(crime_with_blocks['primary_type'] != 'HOMICIDE')].pseudo_pov_rate.describe()

count    522809.000000
mean          0.251785
std           0.144222
min           0.004437
25%           0.129438
50%           0.237299
75%           0.348978
max           0.736909
Name: pseudo_pov_rate, dtype: float64

In [153]:
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE')].average_time.describe() 

count    1262.000000
mean       41.359278
std         5.078144
min        22.666359
25%        38.094466
50%        41.665387
75%        44.191729
max        60.885762
Name: average_time, dtype: float64

In [154]:
crime_with_blocks[(crime_with_blocks['primary_type'] != 'HOMICIDE')].average_time.describe() 

count    522809.000000
mean         39.227229
std           6.336900
min          20.351382
25%          35.408228
50%          39.954568
75%          43.565369
max          60.885762
Name: average_time, dtype: float64

In [155]:
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE')].B19051_001E.describe()

count    1262.000000
mean     1241.383518
std       615.871543
min       200.000000
25%       760.000000
50%      1176.000000
75%      1614.000000
max      4720.000000
Name: B19051_001E, dtype: float64

In [156]:
crime_with_blocks[(crime_with_blocks['primary_type'] != 'HOMICIDE')].B19051_001E.describe()

count    525950.000000
mean       1600.068567
std        1288.670484
min           0.000000
25%         872.000000
50%        1361.000000
75%        1872.000000
max       11012.000000
Name: B19051_001E, dtype: float64

Answer 2 part 2: we can see that for the primary type homicide for both years, that in general this results are consistent with the presented in the previous part where we can see that block were a homicide ocurrs are more poor, have less income and they have to travel longest time to work. This implies that those blocks were this kind of crimes ocurr are more poor.

### Q2-p3

#### Homicide

##### Homicide by tracks by poverty, income and transportation time to work


In [190]:
#Descriptive stats of homicide in 2018 for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE') & (crime_with_blocks['year']=='2018')].pseudo_pov_rate.describe()


count    586.000000
mean       0.334290
std        0.134652
min        0.026226
25%        0.237761
50%        0.337407
75%        0.418401
max        0.718363
Name: pseudo_pov_rate, dtype: float64

In [170]:
#Descriptive stats of homicide in 2017 for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE') & (crime_with_blocks['year']=='2017')].pseudo_pov_rate.describe()


count    676.000000
mean       0.315779
std        0.128731
min        0.026727
25%        0.218448
50%        0.315394
75%        0.398927
max        0.736909
Name: pseudo_pov_rate, dtype: float64

In [171]:
#Descriptive stats of homicide in 2018 for average time
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE') & (crime_with_blocks['year']=='2018')].average_time.describe()


count    586.000000
mean      41.595395
std        5.281571
min       22.666359
25%       38.323428
50%       42.125261
75%       44.484448
max       55.613208
Name: average_time, dtype: float64

In [172]:
#Descriptive stats of homicide in 2017 for average time
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE') & (crime_with_blocks['year']=='2017')].average_time.describe()

count    676.000000
mean      41.154598
std        4.889678
min       26.207294
25%       37.789256
50%       41.483645
75%       44.020678
max       60.885762
Name: average_time, dtype: float64

In [174]:
#Descriptive stats of homicide in 2018 for earnings
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE') & (crime_with_blocks['year']=='2018')].B19051_001E.describe()

count     586.000000
mean     1200.747440
std       606.872279
min       296.000000
25%       743.000000
50%      1122.500000
75%      1601.000000
max      4720.000000
Name: B19051_001E, dtype: float64

In [175]:
#Descriptive stats of homicide in 2017 for earnings
crime_with_blocks[(crime_with_blocks['primary_type'] == 'HOMICIDE') & (crime_with_blocks['year']=='2017')].B19051_001E.describe()

count     676.000000
mean     1276.609467
std       621.868178
min       200.000000
25%       799.000000
50%      1210.000000
75%      1629.000000
max      4213.000000
Name: B19051_001E, dtype: float64

#### Rate of arrest for homicide

In [179]:
#rate of arrest by homicide in 2018
crime_2018_geo_hom = crime_with_blocks[(crime_with_blocks['year']  == '2018') & (crime_with_blocks['primary_type'] == 'HOMICIDE')]
tab8_2018 = pd.crosstab(index=crime_2018_geo_hom['primary_type'], columns = crime_2018_geo_hom['arrest'], margins=True)
tab8_2018.columns = ["Non-Arrested","Arrested", "Total"]
tab8_2018 = tab8_2018/tab8_2018.loc["All","Total"]
tab8_2018 = tab8_2018.div(tab8_2018["Total"], axis=0)
tab8_2018.sort_values("Arrested",ascending=False)

,Non-Arrested,Arrested,Total
primary_type,,,
HOMICIDE,0.730375,0.269625,1.0
All,0.730375,0.269625,1.0


In [181]:
#rate of arrest by homicide in 2017
crime_2017_geo_hom = crime_with_blocks[(crime_with_blocks['year']  == '2017') & (crime_with_blocks['primary_type'] == 'HOMICIDE')]
tab8_2017 =pd.crosstab(index=crime_2017_geo_hom['primary_type'], columns = crime_2017_geo_hom['arrest'], margins=True)
tab8_2017.columns = ["Non-Arrested","Arrested", "Total"]
tab8_2017 = tab8_2017/tab8_2017.loc["All","Total"]
tab8_2017 = tab8_2017.div(tab8_2017["Total"], axis=0)
tab8_2017.sort_values("Arrested",ascending=False)


,Non-Arrested,Arrested,Total
primary_type,,,
HOMICIDE,0.748521,0.251479,1.0
All,0.748521,0.251479,1.0


#### Number of Homicides in the top 10 neighborhoods with homicides

In [182]:
#top 10 communities by number of homicides in 2017
tab9_2017 = crime_with_blocks[(crime_with_blocks['year']  == '2017')& (crime_with_blocks['primary_type'] == 'HOMICIDE')].groupby(['community'])['arrest'].count().nlargest(10)
tab9_2017

community
AUSTIN                    82
NORTH LAWNDALE            43
ROSELAND                  36
SOUTH SHORE               33
HUMBOLDT PARK             30
ENGLEWOOD                 29
AUBURN GRESHAM            28
BRIGHTON PARK             23
GREATER GRAND CROSSING    22
EAST GARFIELD PARK        21
Name: arrest, dtype: int64

In [184]:
#top 10 communities by number of homicides in 2017
tab9_2018 = crime_with_blocks[(crime_with_blocks['year']  == '2018')& (crime_with_blocks['primary_type'] == 'HOMICIDE')].groupby(['community'])['arrest'].count().nlargest(10)
tab9_2018

community
AUSTIN                56
ENGLEWOOD             33
NORTH LAWNDALE        32
WEST GARFIELD PARK    26
WEST PULLMAN          26
SOUTH SHORE           25
AUBURN GRESHAM        23
NEW CITY              23
WEST ENGLEWOOD        23
EAST GARFIELD PARK    22
Name: arrest, dtype: int64

In [183]:
#top 10 communities by number of homicides arrest in 2017
tab10_2017 = crime_with_blocks[(crime_with_blocks['year']  == '2017')& (crime_with_blocks['primary_type'] == 'HOMICIDE')].groupby(['community'])['arrest'].sum().nlargest(10)
tab10_2017

community
AUSTIN            22.0
NORTH LAWNDALE    14.0
SOUTH SHORE       11.0
BRIGHTON PARK      7.0
NEAR WEST SIDE     6.0
WEST ENGLEWOOD     6.0
WOODLAWN           6.0
ENGLEWOOD          5.0
HUMBOLDT PARK      5.0
NEW CITY           5.0
Name: arrest, dtype: float64

In [185]:
#top 10 communities by number of homicides arrest in 2018
tab10_2018 = crime_with_blocks[(crime_with_blocks['year']  == '2018')& (crime_with_blocks['primary_type'] == 'HOMICIDE')].groupby(['community'])['arrest'].sum().nlargest(10)
tab10_2018

community
NORTH LAWNDALE            10.0
AUSTIN                     9.0
BELMONT CRAGIN             7.0
GREATER GRAND CROSSING     7.0
ROSELAND                   7.0
SOUTH SHORE                7.0
WEST ENGLEWOOD             6.0
CHATHAM                    5.0
CHICAGO LAWN               5.0
ENGLEWOOD                  5.0
Name: arrest, dtype: float64

ANS respecto to Homicide: We dont see great variation over time taking the data from ACS survey. This make sense because we dont have any variation in the ACS data due to the source. When we see rate of arrest of a homicide case we don't see mucha difference between year 2018 and 2017. Finally when we compare in which neigborhoods the homicide ocurrs we see some differences. In 2018 the homicide increases in 90 cases in respect to 2017 and the number of murders with some arrest diminish from 82 to 56. Part of the increase is related to the neighborhood of austin and north lawdale.

#### Battery

In [191]:
#Descriptive stats of battery in 2018 for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY') & (crime_with_blocks['year'] == '2018')].pseudo_pov_rate.describe()

count    49428.000000
mean         0.280820
std          0.142766
min          0.004437
25%          0.164809
50%          0.275822
75%          0.380064
max          0.736909
Name: pseudo_pov_rate, dtype: float64

In [192]:
#Descriptive stats of battery in 2017 for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY') & (crime_with_blocks['year'] == '2017')].pseudo_pov_rate.describe()


count    48857.000000
mean         0.284376
std          0.143608
min          0.004437
25%          0.166633
50%          0.280150
75%          0.388186
max          0.736909
Name: pseudo_pov_rate, dtype: float64

In [194]:
#Descriptive stats of battery in 2018 for transport time
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY') & (crime_with_blocks['year'] == '2018')].average_time.describe()


count    49428.000000
mean        40.384336
std          5.926920
min         20.351382
25%         36.881443
50%         40.953398
75%         43.977612
max         60.885762
Name: average_time, dtype: float64

In [195]:
#Descriptive stats of battery in 2017 for transport time
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY') & (crime_with_blocks['year'] == '2017')].average_time.describe()


count    48857.000000
mean        40.485742
std          5.871421
min         20.351382
25%         37.092444
50%         40.995096
75%         43.986920
max         60.885762
Name: average_time, dtype: float64

In [196]:
#Descriptive stats of battery in 2018 for average earnings
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY') & (crime_with_blocks['year'] == '2018')].B19051_001E.describe()

count    49597.000000
mean      1460.001714
std       1081.084763
min          0.000000
25%        816.000000
50%       1275.000000
75%       1812.000000
max      11012.000000
Name: B19051_001E, dtype: float64

In [197]:
#Descriptive stats of battery in 2017 for average earnings
crime_with_blocks[(crime_with_blocks['primary_type'] == 'BATTERY') & (crime_with_blocks['year'] == '2017')].B19051_001E.describe()

count    49020.000000
mean      1442.989841
std       1082.441463
min          0.000000
25%        816.000000
50%       1272.000000
75%       1788.000000
max      11012.000000
Name: B19051_001E, dtype: float64

There is no difference in the result comparing by ACS data between years.

#### Rate of arrest for battery

In [200]:
#rate of arrest by battery in 2018
crime_2018_geo_hom = crime_with_blocks[(crime_with_blocks['year']  == '2018') & (crime_with_blocks['primary_type'] == 'BATTERY')]
tab11_2018 = pd.crosstab(index=crime_2018_geo_hom['primary_type'], columns = crime_2018_geo_hom['arrest'], margins=True)
tab11_2018.columns = ["Non-Arrested","Arrested", "Total"]
tab11_2018 = tab11_2018/tab11_2018.loc["All","Total"]
tab11_2018 = tab11_2018.div(tab11_2018["Total"], axis=0)
tab11_2018.sort_values("Arrested",ascending=False)

,Non-Arrested,Arrested,Total
primary_type,,,
BATTERY,0.79529,0.20471,1.0
All,0.79529,0.20471,1.0


In [204]:
#rate of arrest by battery in 2017
crime_2017_geo_hom = crime_with_blocks[(crime_with_blocks['year']  == '2017') & (crime_with_blocks['primary_type'] == 'BATTERY')]
tab11_2017 = pd.crosstab(index=crime_2017_geo_hom['primary_type'], columns = crime_2017_geo_hom['arrest'], margins=True)
tab11_2017.columns = ["Non-Arrested","Arrested", "Total"]
tab11_2017 = tab11_2017/tab11_2017.loc["All","Total"]
tab11_2017 = tab11_2017.div(tab11_2017["Total"], axis=0)
tab11_2017.sort_values("Arrested",ascending=False)

,Non-Arrested,Arrested,Total
primary_type,,,
BATTERY,0.795022,0.204978,1.0
All,0.795022,0.204978,1.0


#### Number of battery in the top 10 neighborhoods with battery

In [209]:
#top 10 communities by number of battery in 2017
tab12_2017 = crime_with_blocks[(crime_with_blocks['year']  == '2017')& (crime_with_blocks['primary_type'] == 'BATTERY')].groupby(['community'])['arrest'].count().nlargest(10)
tab12_2017

community
AUSTIN                    3290
SOUTH SHORE               2154
NORTH LAWNDALE            2016
AUBURN GRESHAM            1681
WEST ENGLEWOOD            1625
GREATER GRAND CROSSING    1584
HUMBOLDT PARK             1566
ENGLEWOOD                 1482
ROSELAND                  1448
NEAR NORTH SIDE           1386
Name: arrest, dtype: int64

In [208]:
#top 10 communities by number of battery in 2018
tab12_2018 = crime_with_blocks[(crime_with_blocks['year']  == '2018')& (crime_with_blocks['primary_type'] == 'BATTERY')].groupby(['community'])['arrest'].count().nlargest(10)
tab12_2018

community
AUSTIN                    3313
SOUTH SHORE               2024
NORTH LAWNDALE            1928
WEST ENGLEWOOD            1662
AUBURN GRESHAM            1656
HUMBOLDT PARK             1620
ENGLEWOOD                 1564
NEAR NORTH SIDE           1563
GREATER GRAND CROSSING    1540
ROSELAND                  1465
Name: arrest, dtype: int64

In [207]:
#top 10 communities by number of battery arrest in 2017
tab13_2017 = crime_with_blocks[(crime_with_blocks['year']  == '2017')& (crime_with_blocks['primary_type'] == 'BATTERY')].groupby(['community'])['arrest'].sum().nlargest(10)
tab13_2017

community
AUSTIN                    612.0
SOUTH SHORE               450.0
NEAR NORTH SIDE           401.0
AUBURN GRESHAM            355.0
NORTH LAWNDALE            348.0
ROSELAND                  333.0
GREATER GRAND CROSSING    329.0
WEST ENGLEWOOD            297.0
CHATHAM                   285.0
ENGLEWOOD                 269.0
Name: arrest, dtype: float64

In [210]:
#top 10 communities by number of battery arrest in 2018
tab13_2018 = crime_with_blocks[(crime_with_blocks['year']  == '2018')& (crime_with_blocks['primary_type'] == 'BATTERY')].groupby(['community'])['arrest'].sum().nlargest(10)
tab13_2018

community
AUSTIN                    605.0
SOUTH SHORE               437.0
NEAR NORTH SIDE           409.0
AUBURN GRESHAM            379.0
NORTH LAWNDALE            341.0
GREATER GRAND CROSSING    336.0
WEST ENGLEWOOD            328.0
ROSELAND                  319.0
CHATHAM                   307.0
HUMBOLDT PARK             289.0
Name: arrest, dtype: float64

In general we don't see any big difference between years in battery in any variable.

### Q2-p4

In [211]:
#Descriptive stats of deceptive practice for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'DECEPTIVE PRACTICE')].pseudo_pov_rate.describe()

count    35012.000000
mean         0.196544
std          0.132547
min          0.004437
25%          0.092195
50%          0.157025
75%          0.280386
max          0.736909
Name: pseudo_pov_rate, dtype: float64

In [212]:
#Descriptive stats of deceptive practice for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'SEX OFFENSE')].pseudo_pov_rate.describe()

count    1988.000000
mean        0.237314
std         0.143708
min         0.004437
25%         0.122753
50%         0.211667
75%         0.339912
max         0.736909
Name: pseudo_pov_rate, dtype: float64

In [213]:
#Descriptive stats of deceptive practice for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'DECEPTIVE PRACTICE')].B19051_001E.describe()

count    35521.000000
mean      2007.739309
std       1706.024030
min          0.000000
25%       1013.000000
50%       1576.000000
75%       2416.000000
max      11012.000000
Name: B19051_001E, dtype: float64

In [214]:
#Descriptive stats of deceptive practice for poverty
crime_with_blocks[(crime_with_blocks['primary_type'] == 'SEX OFFENSE')].B19051_001E.describe()

count     1998.000000
mean      1632.624625
std       1366.452911
min          0.000000
25%        928.000000
50%       1380.500000
75%       1904.000000
max      11012.000000
Name: B19051_001E, dtype: float64

Reviewing this two statistics we can see that deceptive practice is a crime commited in places of higher income than sex offense, we can see the poverty rate that have some difference but the more differece came from the difference in earning.

## Question 3

### Q3-p1

### Q3-p2

In [256]:
#Extracting data
df_ward43 = crime_with_blocks[(crime_with_blocks["ward"] == "43")]
df_w_w34_2018 = df_ward43[(df_ward43['date'] > '2018-07-19') & (df_ward43['date'] < '2018-07-27')]
df_m_w34_2018 = df_ward43[(df_ward43['date'] > '2018-06-27') & (df_ward43['date'] < '2018-07-27')]
df_w_w34_2017 = df_ward43[(df_ward43['date'] > '2017-07-19') & (df_ward43['date'] < '2017-07-27')]
df_m_w34_2017 = df_ward43[(df_ward43['date'] > '2017-06-27') & (df_ward43['date'] < '2017-07-27')]
df_m_w34 = df_m_w34_2017.append(df_m_w34_2018)
df_w_w34 = df_w_w34_2017.append(df_w_w34_2018)
#filters
battery_filter_w43y = (df_ward43['primary_type'] == 'BATTERY')
battery_filter_w43m = (df_m_w34['primary_type'] == 'BATTERY')
battery_filter_w43w = (df_w_w34['primary_type'] == 'BATTERY')
bulglary_filter_w43y = (df_ward43['primary_type'] == 'BURGLARY')
bulglary_filter_w43m = (df_m_w34['primary_type'] == 'BURGLARY')
bulglary_filter_w43w = (df_w_w34['primary_type'] == 'BURGLARY')
mvt_filter_w43y = (df_ward43['primary_type'] == 'MOTOR VEHICLE THEFT')
mvt_filter_w43m = (df_m_w34['primary_type'] == 'MOTOR VEHICLE THEFT')
mvt_filter_w43w = (df_w_w34['primary_type'] == 'MOTOR VEHICLE THEFT')
rob_filter_w43y = (df_ward43['primary_type'] == 'ROBBERY')
rob_filter_w43m = (df_m_w34['primary_type'] == 'ROBBERY')
rob_filter_w43w = (df_w_w34['primary_type'] == 'ROBBERY')

In [244]:
#Number of arrest and calls by year in ward43
tab14_year=pd.crosstab(index=df_ward43["arrest"],columns=df_ward43["year"], margins=True)
print(tab14_year)

year    2017  2018   All
arrest                  
False   3628  3905  7533
True     326   337   663
All     3954  4242  8196


In [240]:
#Number of arrest and calls by month in ward43
tab14_month=pd.crosstab(index=df_m_w34["arrest"],columns=df_m_w34["year"], margins=True)
print(tab14_month)

year    2017  2018  All
arrest                 
False    326   349  675
True      37    41   78
All      363   390  753


In [241]:
#Number of arrest and calls by week in ward43
tab14_week=pd.crosstab(index=df_w_w34["arrest"],columns=df_w_w34["year"], margins=True)
print(tab14_week)

year    2017  2018  All
arrest                 
False    102    99  201
True       7     8   15
All      109   107  216


In [247]:
#Number of battery incidents and  arrests by year in ward43

tab14_battery_y=pd.crosstab(index=df_ward43[battery_filter_w43y].arrest,columns=df_ward43[battery_filter_w43y].year, margins=True)
print(tab14_battery_y)

year    2017  2018  All
arrest                 
False    278   275  553
True      81    67  148
All      359   342  701


In [250]:
#Number of battery incidents and  arrests in the month in ward43
tab14_battery_m=pd.crosstab(index=df_m_w34[battery_filter_w43m].arrest,columns=df_m_w34[battery_filter_w43m].year, margins=True)
print(tab14_battery_m)

year    2017  2018  All
arrest                 
False     31    25   56
True      10    10   20
All       41    35   76


In [252]:
#Number of battery incidents and  arrests in the week in ward43
tab14_battery_w=pd.crosstab(index=df_w_w34[battery_filter_w43w].arrest,columns=df_w_w34[battery_filter_w43w].year, margins=True)
print(tab14_battery_w)

year    2017  2018  All
arrest                 
False     16     9   25
True       1     2    3
All       17    11   28


In [254]:
#Number of burglary incidents and  arrests by year in ward43
tab14_burglary_y=pd.crosstab(index=df_ward43[bulglary_filter_w43y].arrest,columns=df_ward43[bulglary_filter_w43y].year, margins=True)
print(tab14_burglary_y)

year    2017  2018  All
arrest                 
False    153   175  328
True      11     9   20
All      164   184  348


In [258]:
#Number of burglary incidents and  arrests by month in ward43
tab14_burglary_m=pd.crosstab(index=df_m_w34[bulglary_filter_w43m].arrest,columns=df_m_w34[bulglary_filter_w43m].year, margins=True)
print(tab14_burglary_m)

year    2017  2018  All
arrest                 
False     15    12   27
True       1     2    3
All       16    14   30


In [260]:
#Number of burglary incidents and  arrests by week in ward43
tab14_burglary_w=pd.crosstab(index=df_w_w34[bulglary_filter_w43w].arrest,columns=df_w_w34[bulglary_filter_w43w].year, margins=True)
print(tab14_burglary_w)

year    2017  2018  All
arrest                 
False      3     2    5
True       0     1    1
All        3     3    6


In [261]:
#Number of Motor Vehicule Theft incidents and  arrests by year in ward43
tab14_mvt_y=pd.crosstab(index=df_ward43[mvt_filter_w43y].arrest,columns=df_ward43[mvt_filter_w43y].year, margins=True)
print(tab14_mvt_y)

year    2017  2018  All
arrest                 
False    149   129  278
True      12    10   22
All      161   139  300


In [263]:
#Number of Motor Vehicule Theft incidents and  arrests by month in ward43
tab14_mvt_m=pd.crosstab(index=df_m_w34[mvt_filter_w43m].arrest,columns=df_m_w34[mvt_filter_w43m].year, margins=True)
print(tab14_mvt_m)

year    2017  2018  All
arrest                 
False      6    10   16
All        6    10   16


In [265]:
#Number of Motor Vehicule Theft incidents and  arrests by week in ward43
tab14_mvt_w=pd.crosstab(index=df_w_w34[mvt_filter_w43w].arrest,columns=df_w_w34[mvt_filter_w43w].year, margins=True)
print(tab14_mvt_w)

year    2017  2018  All
arrest                 
False      3     2    5
All        3     2    5


In [267]:
#Number of Robbery incidents and  arrests by year in ward43
tab14_rob_y=pd.crosstab(index=df_ward43[rob_filter_w43y].arrest,columns=df_ward43[rob_filter_w43y].year, margins=True)
print(tab14_rob_y)

year    2017  2018  All
arrest                 
False    122   121  243
True      18    15   33
All      140   136  276


In [268]:
#Number of Robbery incidents and  arrests by month in ward43
tab14_rob_m=pd.crosstab(index=df_m_w34[rob_filter_w43m].arrest,columns=df_m_w34[rob_filter_w43m].year, margins=True)
print(tab14_rob_m)

year    2017  2018  All
arrest                 
False     14     8   22
True       3     1    4
All       17     9   26


In [269]:
#Number of Robbery incidents and  arrests by week in ward43
tab14_rob_w=pd.crosstab(index=df_w_w34[rob_filter_w43w].arrest,columns=df_w_w34[rob_filter_w43w].year, margins=True)
print(tab14_rob_w)

year    2017  2018  All
arrest                 
False      2     3    5
True       0     1    1
All        2     4    6


ANS B: As the statistics are wrong in the sign of the change of type for the most of the different categories implies that the conclusion are missleading.

### Q3-p3

1- we see that the most of the violent crime are concentrated in low income neighborhoods, that implies that are necessary to create and design especific programs to deal with this issue. 
2- Richer and poorest do different crimes, for example we see that the Deceptive Practice crime it's commited in high income sectors of the cities but sex offence or homicide are commited in low income neigborhood. Then it's necessary  to have a police force that have specialized units to deal with this different types of crime. Also, this would imply work with the communities.
3- As we see that the arrest rate of different varies acording the differnt type of crime, we don't have access to conviction statistics, but it will be important to improve the arrest rate of different crimes that are violent.
4- Crimes are concentrated in certain neighborhoods that means that we can focus the effort to diminish them if the efforts are concentrated in them.
5-

### Q3-p4

This analysis dont incorpore more geografical variables provided by the census that would help to understand in which circunstance the different crimes are commited, but with 

## Q4

### Q4-p1

In [289]:
cord = cg.onelineaddress('2111 S Michigan Ave, Chicago, IL')
tract = cord[0]['geographies']['Census Tracts'][0]['TRACT']
filter_tract = (crime_with_blocks['tractce10'] == tract)

In [291]:
df_crime_q4p1 = crime_with_blocks[filter_tract]

In [309]:
#Number of arrest both years in the tract of the direction
tab15=pd.crosstab(index=df_crime_q4p1["primary_type"], columns=['Total'], margins=True)
#% of crime in the tract
tab15/tab15.loc['All']

col_0,Total,All
primary_type,,
ARSON,0.001171,0.001171
ASSAULT,0.064793,0.064793
BATTERY,0.153396,0.153396
BURGLARY,0.024590,0.024590
CONCEALED CARRY LICENSE VIOLATION,0.000390,0.000390
CRIM SEXUAL ASSAULT,0.005855,0.005855
CRIMINAL DAMAGE,0.079625,0.079625
CRIMINAL TRESPASS,0.040203,0.040203
DECEPTIVE PRACTICE,0.154567,0.154567


ANS: It would be  31% the theft probability

### Q4-p2

In [349]:
df_crime_q4p2 = crime_with_blocks[crime_with_blocks['primary_type'] == 'THEFT']
n_theft = len(df_crime_q4p2)
theft_uptown = len(df_crime_q4p2[df_crime_q4p2['area_num_1'] == '3'])
p_thef_uptown = theft_uptown/n_theft
theft_gp = len(df_crime_q4p2[df_crime_q4p2['area_num_1'] == '26']) + len(df_crime_q4p2[df_crime_q4p2['area_num_1'] == '27'])
p_thef_gp = theft_gp/n_theft

In [351]:
p_thef_gp

0.019446386900599497

In [352]:
p_thef_uptown

0.015124967589355165

ANS: it's most likely that would be from Garfield Park, because the probability is 1.9% instead uptown is 1.5%.

### Q4-p3

As there are 160 calls for battery from garfield park and 100 from uptown, implies that the probabilities are 16% and 10% respectively. This means that is more likely to came from garfield park by 6%.